In [18]:
import h2o
from h2o.estimators.pca import H2OPrincipalComponentAnalysisEstimator as pca

In [3]:
h2o.init(nthreads = -1, max_mem_size = 4)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.4" 2019-07-16; OpenJDK Runtime Environment (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3); OpenJDK 64-Bit Server VM (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3, mixed mode, sharing)
  Starting server from /home/linuxbrew/.linuxbrew/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpk6q37vg8
  JVM stdout: /tmp/tmpk6q37vg8/h2o_frn_started_from_python.out
  JVM stderr: /tmp/tmpk6q37vg8/h2o_frn_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Europe/Madrid
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.11
H2O cluster version age:,2 days
H2O cluster name:,H2O_from_python_frn_1wbdac
H2O cluster total nodes:,1
H2O cluster free memory:,4 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [62]:
titanic= h2o.import_file('titanic.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [66]:
y_columns = "Survived"
x_columns = ["Pclass","Sex","Fare", "Embarked"]

In [45]:
titanic[y_columns]

Survived
0
1
1
1
0
0
0
0
1
1


In [64]:
titanic[y_columns] = titanic[y_columns].asfactor()
titanic[y_columns].levels()

[['0', '1']]

In [59]:
train, test=titanic.split_frame(ratios = [.8])
X_train=train[x_columns]
y_train=train[y_columns]
X_test=test[x_columns]
y_test=test[y_columns]

In [48]:
X_train.pca=pca(k = 3, transform = "STANDARDIZE", pca_method="Power",
                   use_all_factor_levels=True, impute_missing=True)
X_train.pca.train(training_frame=X_train)

pca Model Build progress: |███████████████████████████████████████████████| 100%


In [49]:
X_train.pca.varimp(use_pandas=True)

,,pc1,pc2,pc3
0,Standard deviation,1.271104,1.096803,0.667007
1,Proportion of Variance,0.403645,0.300534,0.111147
2,Cumulative Proportion,0.403645,0.704179,0.815327


In [50]:
X_train.pca.rotation()


Rotation: 


,,pc1,pc2,pc3
0,Embarked.C,-0.076889,0.105578,0.006421
1,Embarked.Q,0.047299,0.022529,0.073346
2,Embarked.S,0.137081,0.725805,0.129344
3,Sex.female,-0.056160,0.250055,0.524851
4,Sex.male,0.163652,0.603858,-0.315741
5,Pclass,0.682575,-0.151639,0.594195
6,Fare,-0.690808,0.106711,0.499620


In [55]:
from h2o.estimators.random_forest import H2ORandomForestEstimator as rf

In [68]:
rf_fit1 = rf(model_id='rf_fit1', seed=1)
rf_fit1.train(x=X_columns, y=y_columns, training_frame=train)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [73]:
rf_fit2 = rf(model_id='rf_fit2', ntrees=500, seed=1)
rf_fit2.train(x=X_columns, y=y_columns, training_frame=train)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [74]:
rf_perf1 = rf_fit1.model_performance(test)
rf_perf2 = rf_fit2.model_performance(test)

In [76]:
print(rf_perf1.auc())
print(rf_perf2.auc())

0.8404761904761905
0.843421052631579


In [77]:
rf_fit1.model_performance

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  rf_fit1


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,50.0,50.0,64065.0,12.0,20.0,15.0,55.0,121.0,97.36




ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.14850278168757383
RMSE: 0.3853605865777841
LogLoss: 0.7037666403896635
Mean Per-Class Error: 0.21051808654496285
AUC: 0.839393171061528
pr_auc: 0.7527193234094504
Gini: 0.678786342123056

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3700657894736842: 


,,0,1,Error,Rate
0,0,359.0,76.0,0.1747,(76.0/435.0)
1,1,67.0,205.0,0.2463,(67.0/272.0)
2,Total,426.0,281.0,0.2023,(143.0/707.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2019-12-08 19:22:35,0.001 sec,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,,2019-12-08 19:22:35,0.029 sec,1.0,0.433676,4.717028,0.788602,0.255140,2.025897,0.230453
2,,2019-12-08 19:22:35,0.042 sec,2.0,0.417265,3.857136,0.794420,0.400799,2.073223,0.220624
3,,2019-12-08 19:22:35,0.058 sec,3.0,0.425648,4.167162,0.779683,0.392062,2.069318,0.211650
4,,2019-12-08 19:22:35,0.076 sec,4.0,0.424949,3.554895,0.788498,0.436392,2.140571,0.224490
5,,2019-12-08 19:22:35,0.089 sec,5.0,0.406359,2.838284,0.814161,0.458315,2.245967,0.192429
6,,2019-12-08 19:22:35,0.103 sec,6.0,0.411784,2.860249,0.808024,0.449637,2.214188,0.229135
7,,2019-12-08 19:22:35,0.115 sec,7.0,0.411133,2.781481,0.808745,0.450978,2.221190,0.196769
8,,2019-12-08 19:22:35,0.126 sec,8.0,0.409890,2.307929,0.809249,0.516499,2.240745,0.201460
9,,2019-12-08 19:22:35,0.139 sec,9.0,0.403535,1.998974,0.818208,0.533339,2.313980,0.227994



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,Sex,2213.242676,1.000000,0.479160
1,Fare,1748.678101,0.790098,0.378584
2,Pclass,481.038940,0.217346,0.104143
3,Embarked,176.042603,0.079541,0.038113


<bound method ModelBase.model_performance of >

In [83]:
rf_fit3 = rf(model_id='rf_fit3', seed=1, nfolds=5, ntrees=2000)
rf_fit3.train(x=x_columns, y=y_columns, training_frame=train)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [84]:
rf_perf3 = rf_fit3.model_performance(test)
print(rf_perf3.auc())

0.843421052631579
